In [ ]:
import tensorflow as tf
import numpy as np
from google.colab import drive

drive.mount('/content/drive')  # هذا السطر لربط Google Drive ببيئة Colab

# هنا تضع المسار الصحيح لبياناتك داخل Google Drive
base_dir = '/content/drive/My Drive/path_to_your_data'
IMAGE_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 97

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

base_model = tf.keras.applications.MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # لربط Google Drive

from tensorflow import keras
import os
from keras.preprocessing import image
import numpy as np
import tensorflow as tf

# تحديد مسار المجلد الرئيسي داخل Google Drive
base_dir = '/content/drive/My Drive/plant'  # تأكد من تعديل هذا المسار لمسار بياناتك الحقيقي

# جمع أسماء الفولدرات
class_names = sorted(os.listdir(base_dir))  # تأكد إن الفولدر مافيهوش حاجة غير الفولدرات بتاعة النباتات

# تحميل النموذج
model_path = '/content/drive/My Drive/plant_model.h5'  # تعديل المسار لمكان حفظ الموديل
model = tf.keras.models.load_model(model_path)

# كود لرفع الصورة من جهازك
from google.colab import files
uploaded = files.upload()

# تحديد اسم الصورة المرفوعة
for fn in uploaded.keys():
  img_path = fn
  break

img = image.load_img(img_path, target_size=(224, 224))  # استخدام نفس الحجم الذي تم تدريب النموذج عليه

# تحويل الصورة لمصفوفة وتعديل الألوان
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# عمل تنبؤ
predictions = model.predict(img_array)

# طباعة الفئة الأكثر احتمال
predicted_class = class_names[np.argmax(predictions)]
print(f'النبات المتنبأ به: {predicted_class}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving WhatsApp Image 2023-12-16 at 10.43.50 AM.jpeg to WhatsApp Image 2023-12-16 at 10.43.50 AM.jpeg
1/1 [==============================] - 3s 3s/step
النبات المتنبأ به: calendula
